In [1]:
import findspark
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

In [2]:
try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

Just created a SparkContext


In [3]:
sc.master

'local[4]'

In [4]:
pwd

'/home/jovyan'

In [6]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/home/jovyan/small_english_tweets.csv')

In [15]:
df.show(10)

+--------------------+--------------------+---------------+--------------------+------------------+----------------+-------------+------------+------------------+---------------+--------+----+
|                 _c0|          created_at|    screen_name|                text|            source|favourites_count|retweet_count|country_code|   place_full_name|followers_count|verified|lang|
+--------------------+--------------------+---------------+--------------------+------------------+----------------+-------------+------------+------------------+---------------+--------+----+
|            12580835|2020-03-26T04:01:14Z|     AnnaKealan|hoping for more c...|Twitter for iPhone|             0.0|          0.0|        null|              null|             10|   False|  en|
|              590296|2020-03-12T04:04:07Z|   DrPaulRLong1|Is Trump even con...|              null|            null|         null|        null|              null|           null|    null|null|
|#CoronavirusPande...|             

In [16]:
(train_set, val_set, test_set) = df.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [17]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])



Py4JJavaError: An error occurred while calling o166.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 11.0 failed 1 times, most recent failure: Lost task 3.0 in stage 11.0 (TID 26, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection